In [2]:
import pandas as pd
import numpy as np

In [3]:
test_file = 'symbol_prediction_test.csv'

In [4]:
test_df = pd.read_csv(test_file, sep=',', engine='c')

In [5]:
test_df.head()

,id,text
0,1,"""Take care, Lenox,"" exclaimed Zaidie, taking a..."
1,2,We followed him. When the three of us were bac...
2,3,"Ten years ago Lavalle, ""that imperturbable dre..."
3,4,"""I think I'd rather se■ what the rings are lik..."
4,5,"Surprisingly, she did shut up. Gordon went to ..."


In [75]:
test_df['text'][33]

'"Larr■," I said gently, "I was there."'

In [6]:
missing_symbol = '■'

In [7]:
from gensim.models import Word2Vec

In [10]:
import hunspell
spellchecker = hunspell.HunSpell('hunspell/en_GB-large.dic', 'hunspell/en_GB-large.aff')

In [70]:
spellchecker.suggest('Larr■')[0]

'Larry'

In [54]:
print(spellchecker.suggest('se■'))
print(test_df['text'][3].find(missing_symbol))
print(test_df['text'][3].find(' ', 22))
print(test_df['text'][3].rfind(' ', 0, 22))
print(test_df['text'][3][19:24].strip())
# print(test_df['text'][3][19:24].strip(), ':', spellchecker.spell(test_df['text'][3][19:24].strip()))
print(test_df['text'][3][19:22].strip(), ':', spellchecker.spell(test_df['text'][3][19:22].strip()))
print(test_df['text'][3][23:24], ':', spellchecker.spell(test_df['text'][3][23:24]))

['see', 'sea', 'sen', 'ser', 'set', 'sec', 'sem', 'sew']
22
23
19
se■
se : False
  : True


In [ ]:
space_symbol = ' '
empty_symbol = ''
indices = []
predicted_symbols = []
for index, line in enumerate(test_df['text']):
    indices.append(index+1)
    index_of_missing = line.find(missing_symbol)
    index_of_prev_space = line.rfind(space_symbol, 0, index_of_missing)
    index_of_next_space = line.find(space_symbol, index_of_missing+1)
    prev_word = line[index_of_prev_space:index_of_missing].strip()
    next_word = line[index_of_missing+1:index_of_next_space+1].strip()
    # if missing_symbol is a delimiter - store ' ' as a prediction
    if (prev_word and spellchecker.spell(prev_word)) and (next_word and spellchecker.spell(next_word)):
        predicted_symbols.append(space_symbol)
        continue
    else:
        word_with_missing_symbol = line[index_of_prev_space:index_of_next_space]
        index_of_missing = word_with_missing_symbol.find(missing_symbol)
        if index_of_missing == -1:
            print('Error at index:', index)
            predicted_symbols.append(space_symbol)
        else:
            all_suggested = spellchecker.suggest(word_with_missing_symbol)
#             print('for', word_with_missing_symbol, ':', )
            if not all_suggested:
                predicted_symbols.append(empty_symbol)
                continue
            suggested = spellchecker.suggest(word_with_missing_symbol)[0]
#             print('at', index, ':', suggested, '| index:', index_of_missing-1)
            if index_of_missing >= len(suggested):
                predicted_symbols.append(empty_symbol)
            else:
                predicted_symbols.append(suggested[index_of_missing-1])

Error at index: 18
Error at index: 20
Error at index: 24
Error at index: 33
Error at index: 61
Error at index: 109
Error at index: 112
Error at index: 122
Error at index: 138
Error at index: 145
Error at index: 185
Error at index: 188
Error at index: 191
Error at index: 204
Error at index: 230
Error at index: 239
Error at index: 243
Error at index: 261
Error at index: 273
Error at index: 284
Error at index: 298
Error at index: 306
Error at index: 360
Error at index: 366
Error at index: 371
Error at index: 380
Error at index: 389
Error at index: 392
Error at index: 410
Error at index: 412
Error at index: 417
Error at index: 430


In [ ]:
predictions = pd.DataFrame({'id': indices, 'y': predicted_symbols})
predictions.to_csv('symbol_prediction_sumb1.csv', sep=',')